In [1]:
import tensorflow as tf
from tensorflow import convert_to_tensor, string
from tensorflow.keras.layers import TextVectorization, Embedding, Layer
from tensorflow.data import Dataset
import numpy as np
import matplotlib.pyplot as plt

import nltk, random
import numpy as np
import pandas as pd

from tensorflow import keras
from sklearn.model_selection import train_test_split



In [2]:
df = pd.read_excel('train.xlsx')
print(df.shape)
df.head()

(1052, 5)


,doc,text,topic,Topic_Keywords,Number_of_words
0,0,linguistic input features improve neural machi...,12,"translation,language,machine,system,source,mod...",3932
1,1,multiresolution recurrent neural networks an a...,0,"system,model,language,evaluation,score,example...",3992
2,2,from captions to visual concepts and back hao ...,7,"visual,feature,image,network,video,object,mult...",4193
3,3,deep reinforcement learning for listwise recom...,3,"user,item,recommendation,model,matrix,system,m...",3989
4,4,sentiment classication using document embeddin...,13,"word,model,text,classiﬁcation,document,sentime...",3835


In [23]:
def tokenize(text):
    '''a function which convert a given text into tokens'''
    return nltk.word_tokenize(text)
df['tokens'] = df['text'].apply(tokenize)
df

,doc,text,topic,Topic_Keywords,Number_of_words,tokens
0,0,linguistic input features improve neural machi...,12,"translation,language,machine,system,source,mod...",3932,"[linguistic, input, features, improve, neural,..."
1,1,multiresolution recurrent neural networks an a...,0,"system,model,language,evaluation,score,example...",3992,"[multiresolution, recurrent, neural, networks,..."
2,2,from captions to visual concepts and back hao ...,7,"visual,feature,image,network,video,object,mult...",4193,"[from, captions, to, visual, concepts, and, ba..."
3,3,deep reinforcement learning for listwise recom...,3,"user,item,recommendation,model,matrix,system,m...",3989,"[deep, reinforcement, learning, for, listwise,..."
4,4,sentiment classication using document embeddin...,13,"word,model,text,classiﬁcation,document,sentime...",3835,"[sentiment, classication, using, document, emb..."
...,...,...,...,...,...,...
1047,1059,selfattentive sequential recommendation wangch...,3,"user,item,recommendation,model,matrix,system,m...",4137,"[selfattentive, sequential, recommendation, wa..."
1048,1060,graphbtm graph enhanced autoencoded variationa...,13,"word,model,text,classiﬁcation,document,sentime...",4166,"[graphbtm, graph, enhanced, autoencoded, varia..."
1049,1061,zoho at semevaltask semisupervised domain adap...,13,"word,model,text,classiﬁcation,document,sentime...",2308,"[zoho, at, semevaltask, semisupervised, domain..."
1050,1062,building nontrivial paraphrase corpus using mu...,0,"system,model,language,evaluation,score,example...",3395,"[building, nontrivial, paraphrase, corpus, usi..."


In [24]:
tokens=df['tokens']

# Evaluation Class

In [1]:
from gensim.models import Word2Vec
from scipy import spatial

class Evaluation:
    def __init__(self,  embed_dim=256, tokens=None):
        self.embed_dim = embed_dim
        self.embedding_generator(tokens)
        
        
    def embedding_generator(self, tokens):
        self.w2v_model = Word2Vec(tokens,
                     min_count=1, # min frequency of a word, ignore words with frequency lower than min_count
                     window=10, # maximum distance between the current and predicted word within a sentence
                     vector_size=256, # Dimensionality of the result vector
                     alpha=0.03, # learning rate
                     min_alpha=0.0006, # learning rate will drop during training linearly by min_alpha 
                     workers = 4, # worker threads to train the model
                     seed = 42 # random number generator
                    )

    
    
    def get_seq_embeddings(self, seq):
        embed=np.array([0]*self.embed_dim)
        for i in seq:
            embed=embed+ self.w2v_model.wv[i]
        return embed
        
    def cal_similarity(self, txt1, txt2, n_grams):
        
        seq1=txt1.split(' ')
        seq2=txt2.split(' ')
        
        embed1=self.get_seq_embeddings(seq1[-1*n_grams:])
        embed2=self.get_seq_embeddings(seq2[-1*n_grams:])
        
        return 1-spatial.distance.cosine(embed1, embed2)
    

    def evaluate(self, df, n_grams):
        return df.apply(lambda x: self.cal_similarity(x['y'], x['_y'], n_grams), axis=1).mean()
        
        
    
    

/Users/ghadeeraboalrob/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [107]:
evaluation_obj = Evaluation( 256, tokens)

In [100]:
txt='segmentation that exploits the notion of paradigms which are sets'
txt1='segmentation that exploits the notion of paradigms which are groups'
evaluation_obj.cal_similarity(txt, txt1, 5)

0.9124328301497067

In [108]:
d=pd.DataFrame({'y':['notion of paradigms which are sets of morphological categories', 
     'segmentation that exploits the notion of paradigms which are sets'], 
 '_y':['notion of paradigms which are sets of morphological classes', 
     'segmentation that exploits the notion of paradigms which are groups']})

evaluation_obj.evaluate(d, 5)

0.9373808424118478

## Trials

In [83]:
from scipy import spatial
idxs=[-10, -9, -8, -7, -6, -5, -4, -3, -2]
res=[]
for idx in idxs:
    embed1=get_seq_embeddings(v1[idx:])
    embed2=get_seq_embeddings(v2[idx:])
    res.append(1-spatial.distance.cosine(embed1, embed2))
d=pd.DataFrame({'grams':[10,9,8,7,6,5,4,3,2], 'similarity':res})
d

,grams,similarity
0,10,0.967073
1,9,0.963002
2,8,0.942517
3,7,0.945211
4,6,0.919210
5,5,0.913097
6,4,0.878998
7,3,0.865207
8,2,0.824165


In [70]:
from numpy import dot
from numpy.linalg import norm

result = dot(embed1, embed2)/(norm(embed1)*norm(embed2))
print(result)

0.9192102021480787
